In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import sklearn as sk
import os
import random
import math

In [ ]:
import sys
sys.version

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

print("Num Physical Devices Available: ", len(physical_devices))

print(tf.__version__)
print(cv2.__version__)

**Constants**


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
#PATH = "C:/Python/chen22/hitting_behaviors/video_data/violence_dataset/"
PATH = "video_data/violence_dataset/"

**Make Datatset**


In [ ]:
def video_to_frames(p):
    #assert os.path.exists(p)
    cap = cv2.VideoCapture(p)
    
    #assert cap
    rate = math.floor(cap.get(3))

    ImageFrames = []
    #assert cap.isOpened()
    

    '''make sure to figure out how to extract a certain amount of frames so you don't use too much memory'''
    while cap.isOpened():
        #ID = cap.get(1)
        
        success, image = cap.read()
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(n_frames)
        #assert success

        if success:

            for frame in range(n_frames):
                if frame % 10 == 0:
                    
                    
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    ImageFrames.append(image)
                else: continue
            cap.release()
    #print("successful frame capture")
            return ImageFrames

        else: 
            break

In [ ]:
CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
for category in os.listdir(PATH):
    
    
    path = os.path.join(PATH, category)
    #print(path)
    #assert os.path.exists(path)


    class_num = CATEGORIES.index(category)
    for i, video in tqdm(enumerate(os.listdir(path)[0:350]), total=len(os.listdir(path)[0:350])):

        #print(path + '/' + video)
        #print(os.path.join(path, video))
        frames = video_to_frames(path + '/' + video)
        if frames:
        
            for j, frame in enumerate(frames):
                X_set.append(frame)
                y_set.append(class_num)
        else: continue

            
print("success")

In [ ]:
X_set = np.array(X_set).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_set)
len(X_set), len(y_set)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_set, y_set, test_size=0.2, random_state=42)

X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

In [ ]:
#EffNet = tf.keras.applications.EfficientNetV2B0(include_top=False, weights = 'imageNet', )